# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import importlib

# Force reload of api_keys.py
import api_keys
importlib.reload(api_keys)  # Ensures Python loads the latest version

# Import API key
from api_keys import geoapify_key

# Verify that the API key is correctly loaded
print("Your API Key:", geoapify_key)  # Should display your actual key


Your API Key: 4f18c85a92df4dc6ad15fef9258ac7c9


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,30.03,74,40,3.09,CK,2025-01-25 02:01:56
1,1,uturoa,-16.7333,-151.4333,28.20,74,2,5.68,PF,2025-01-25 02:04:33
2,2,invercargill,-46.4000,168.3500,22.23,54,15,0.45,NZ,2025-01-25 02:00:04
3,3,wailua homesteads,22.0669,-159.3780,23.44,66,100,5.14,US,2025-01-25 02:04:34
4,4,klyuchi,52.2667,79.1667,-19.22,80,100,5.39,RU,2025-01-25 02:04:34


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x="Lon",
    y="Lat",
    size="Humidity",
    color="City",
    cmap="Category20",
    frame_width=800,
    frame_height=500,
    alpha=0.7,
    tiles="OSM",
    hover_cols=["City"]
).opts(
    xlabel="Longitude",
    ylabel="Latitude",
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Humidity"] >= 30) & (city_data_df["Humidity"] <= 50) &
    (city_data_df["Wind Speed"] >= 1) & (city_data_df["Wind Speed"] <=5) &
    (city_data_df["Cloudiness"] >= 0) & (city_data_df["Cloudiness"] <=50)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head(15)

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,87,lodwar,3.1191,35.5973,26.30,47,0,1.38,KE,2025-01-25 02:04:55
98,98,seka,17.9285,103.9552,24.52,38,6,1.82,TH,2025-01-25 02:04:27
116,116,wildwood,38.5828,-90.6629,-0.57,50,0,4.12,US,2025-01-25 02:05:03
139,139,charters towers,-20.1000,146.2667,34.45,36,3,3.78,AU,2025-01-25 01:57:46
146,146,bandarban,22.1965,92.2262,18.11,44,0,1.01,BD,2025-01-25 02:05:10
172,172,at taj,24.2049,23.2857,12.62,39,0,3.37,LY,2025-01-25 02:05:17
197,197,laboulaye,-34.1266,-63.3912,26.07,36,0,3.85,AR,2025-01-25 02:05:08
215,215,san patricio,28.0170,-97.5169,6.97,48,0,3.58,US,2025-01-25 02:05:28
217,217,mandera,3.9366,41.8670,22.82,46,0,2.38,KE,2025-01-25 02:05:29
244,244,minbu,20.1833,94.8833,20.31,45,0,2.04,MM,2025-01-25 02:05:35


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lon", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(15)

,City,Country,Lat,Lon,Humidity,Hotel Name
87,lodwar,KE,3.1191,35.5973,47,
98,seka,TH,17.9285,103.9552,38,
116,wildwood,US,38.5828,-90.6629,50,
139,charters towers,AU,-20.1000,146.2667,36,
146,bandarban,BD,22.1965,92.2262,44,
172,at taj,LY,24.2049,23.2857,39,
197,laboulaye,AR,-34.1266,-63.3912,36,
215,san patricio,US,28.0170,-97.5169,48,
217,mandera,KE,3.9366,41.8670,46,
244,minbu,MM,20.1833,94.8833,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
import requests

# Set parameters to search for a hotel
radius = 10000  # Search within 10 km
base_url = "https://api.geoapify.com/v2/places"

params = {
    "categories": "accommodation.hotel",  # Look for hotels
    "limit": 1,  # Get only the nearest hotel
    "apiKey": "4f18c85a92df4dc6ad15fef9258ac7c9"  # Replace with your actual API key
}

# Print a message to follow up on the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat, lon = row["Lat"], row["Lon"]

    # Create a copy of params to avoid modifying the global dictionary
    query_params = params.copy()
    query_params["filter"] = f"circle:{lon},{lat},{radius}"
    query_params["bias"] = f"proximity:{lon},{lat}"

    # Make an API request using the query_params dictionary
    response = requests.get(base_url, params=query_params)

    # Check if the request was successful
    if response.status_code == 200:
        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_name = name_address["features"][0]["properties"]["name"]
            hotel_df.loc[index, "Hotel Name"] = hotel_name  # Assign correctly
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        print(f"API request failed for {row['City']} (Status Code: {response.status_code})")
        hotel_df.loc[index, "Hotel Name"] = "API request failed"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - Nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(20)


Starting hotel search...
lodwar - Nearest hotel: Lodwar Modern Lodge
seka - Nearest hotel: No hotel found
wildwood - Nearest hotel: Wildwood Hotel
charters towers - Nearest hotel: No hotel found
bandarban - Nearest hotel: Hotel Plaza
at taj - Nearest hotel: No hotel found
laboulaye - Nearest hotel: Hotel Plaza
san patricio - Nearest hotel: No hotel found
mandera - Nearest hotel: Kornesh Hotel and Lodging
minbu - Nearest hotel: Rolex Guesthouse
aswan - Nearest hotel: Oscar Hotel
lamar - Nearest hotel: 1st St Lofts
isna - Nearest hotel: No hotel found
vallenar - Nearest hotel: Hotel del Marqués
ulanhot - Nearest hotel: 阿勒泰大酒店
carrizo springs - Nearest hotel: Texan Inn
ratnagiri - Nearest hotel: Vihar
anar darah - Nearest hotel: No hotel found
sillod - Nearest hotel: No hotel found
udayagiri - Nearest hotel: No hotel found
sur - Nearest hotel: Sur Hotel
plastun - Nearest hotel: Россияночка
tobruk - Nearest hotel: فندق زهرة البطنان -طبرق
bangassou - Nearest hotel: Hotel De Tourisme
pokhara

,City,Country,Lat,Lon,Humidity,Hotel Name
87,lodwar,KE,3.1191,35.5973,47,Lodwar Modern Lodge
98,seka,TH,17.9285,103.9552,38,No hotel found
116,wildwood,US,38.5828,-90.6629,50,Wildwood Hotel
139,charters towers,AU,-20.1000,146.2667,36,No hotel found
146,bandarban,BD,22.1965,92.2262,44,Hotel Plaza
172,at taj,LY,24.2049,23.2857,39,No hotel found
197,laboulaye,AR,-34.1266,-63.3912,36,Hotel Plaza
215,san patricio,US,28.0170,-97.5169,48,No hotel found
217,mandera,KE,3.9366,41.8670,46,Kornesh Hotel and Lodging
244,minbu,MM,20.1833,94.8833,45,Rolex Guesthouse


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points( 
    x="Lon",
    y="Lat",
    size="Humidity",
    color="City",
    cmap="Category20",
    frame_width=800,
    frame_height=500,
    alpha=0.7,
    tiles="OSM",
    hover_cols=["City", "Country", "Hotel Name"]  # Add hotel name & country to hover
).opts(
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Humidity,Country,Hotel Name)